# A Data-Based Understanding of Political Revolutions
Political upheavals have been ever present throughout humanity. Political leaders shape the context for everything we know. For that reason, we as a society need to generate a concrete understanding of when a revolution or political change is imminent. This analysis seeks to forecast whether a given protest will lead to a revolution within 90 days. 

The stakeholders for this analysis are wide reaching, but it is most relevant to political organizers and leadership. They can use this approach to best understand where efforts are best focused and most likely to lead to an impact. 

We as a society have only begun to establish the collection of data necessary to make these analyses feasible, but we have crossed the threshold of significance beyond which there is substantial merit in this project. 

## Notebook Structure
1. xxx
2. xxx
3. xxx
4. xxx




## The Data & Sources
The analysis combines three core datasets from widely different sources to provide a distinctly unique understanding of the subject. They are described below.

#### The Mass Mobilization Project

The first dataset, used as the center of the analysis, is incredibly valuable. It is described in the source documentation as *"an effort to understand citizen movements against governments, what citizens want when they demonstrate against governments, and how governments respond to citizens. The MM data cover 162 countries between 1990 and 2018. These data contain events where 50 or more protesters publicly demonstrate against government, resulting in more than 10,000 protest events. Each event records location, protest size, protester demands, and government responses."* [(1)](https://massmobilization.github.io/about.html)

The project is sponsored by the Political Instability Task Force (PITF). The PITF is funded by the Central Intelligence Agency (CIA). [(1)](https://massmobilization.github.io/about.html)


#### "Regime changes"
[Description]

[Source]


#### "IDB"
[Description]

[Source]

## Technical Notes
Given the large-scale nature of the project, this notebook inevitably does not contain the entire analysis. It does not go into depth on each of the choices made for feature selection or data cleaning. Consider this the "top level" notebook, and those more detail-oriented parts of the analysis are handled in their own spaces. Here is where you should go to find more details:


1. **[protests cleaning file name]:** Refer to this notebook for a ground-up analysis of the "Protests" dataset. It includes important features such as the nature of each protest (size, objectives, location, etc.). Especially exhaustive and detailed data cleaning choices are made in this notebook.
2. **[regime change file name]:** Refer to this notebook for a full study of the "Regime Change" data. Primarily, this data provides the target feature for the entire analysis: regime change. It indicates when are where political change occurs. Interesting feature engineering takes place here.
3. **[idb analysis file name]:** Refer to this notebook for an impressively comprehensive dataset surrounding descriptive attributes of governments around the world over many decades. Important information includes the political system of countries at specified times, the tenure of the country's primary leader, and the percentage of the popular vote the leader received (where relevant). There are descriptors as granular as the total number of seats in congress (where relevant). Extensive feature selection takes place in this notebook.

## Stage I. Data Preparation
[description]

In [1]:
# Basic imporrts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

# Model preprocessing and processing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_selector, make_column_transformer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline
from sklearn.base import clone

# Models
from sklearn.dummy import DummyClassifier
from xgboost import XGBClassifier

# Performance evaluation
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score
from sklearn.metrics import plot_confusion_matrix
import shap

# Display options
pd.options.display.max_columns = 200
%matplotlib inline

# Convenience for working with external .py files
%load_ext autoreload
%autoreload 2

# Global constants
NOT_APLIC_STR = "NA_SS"
NOT_APLIC_NUM = -999.0
RANDOM_STATE = 2021

**Import "Regime Change" dataset**

In [2]:
# Import cleaned database via SQL
engine = create_engine('sqlite:///../data/processed/regime_changes.db')
with engine.begin() as connection:
    df_regimes = pd.read_sql('SELECT * FROM regime_changes', con=connection)

# Type casting
df_regimes.startdate = pd.to_datetime(df_regimes.startdate)
df_regimes.enddate = pd.to_datetime(df_regimes.enddate)

**Import "Protests" dataset**

In [3]:
# Import cleaned database via SQL
engine = create_engine('sqlite:///../data/processed/protests.db')
with engine.begin() as connection:
    df_protests = pd.read_sql('SELECT * FROM protests', con=connection)

# Type casting
df_protests.startdate = pd.to_datetime(df_protests.startdate)

**Import "Government Characterization" dataset**

In [4]:
# Import cleaned database via SQL
engine = create_engine('sqlite:///../data/processed/governments.db')
with engine.begin() as connection:
    df_govts = pd.read_sql('SELECT * FROM governments', con=connection)

# Set index to be used in SQL join
df_govts.index = df_govts.year_scode

# Drop now-duplicated feature
df_govts.drop('year_scode', axis=1, inplace=True)

**Join "Protests" and "Government Characterization" datasets

In [5]:
# Join both dataframes
df = df_protests.join(df_govts, how='left', on='year_scode')

# Remove entries that don't have corresponding 'government' data
#df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15208 entries, 0 to 15207
Data columns (total 73 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   country                            15208 non-null  object        
 1   scode                              15208 non-null  object        
 2   region                             15208 non-null  object        
 3   protestnumber                      15208 non-null  int64         
 4   protesterviolence                  15208 non-null  int64         
 5   startdate                          15208 non-null  datetime64[ns]
 6   duration_days                      15208 non-null  int64         
 7   participants                       15208 non-null  int64         
 8   participants_category              15208 non-null  object        
 9   demand_labor-wage-dispute          15208 non-null  int64         
 10  demand_land-farm-issue            

**Remove features with missing values**

In [ ]:
# Constants used throughout analysis
NA_STRING = "NA_SS"
NA_NUMBER = -999.0


# Relace 'placeholder' NaN values, as defined by data dictionary (see raw data directory)
df.replace(NA_STRING, np.nan, inplace=True)
df.replace(NA_NUMBER, np.nan, inplace=True)
print(df.info())


# Set semi-arbitrary threshold for the maximum number of missing values to justify keeping
MAX_MISSING_VALUES = 250

# Copy main df to retain a full dataset
df_cut = df.copy()

# Determine the number of missing values in each column
for col in df_cut.columns:
    na_ct = df_cut[col].isna().sum()
    
    if na_ct > MAX_MISSING_VALUES:
        df_cut.drop(col, axis=1, inplace=True)
        
df_cut.dropna(inplace=True)
df_cut.info()

## Modeling
[description]

## Evaluation
[description]

## Conclusion
[description]